In [2]:
import os 
import pandas as pd
import requests
import json
import dotenv
import lyricscraper
dotenv.load_dotenv()

True

In [3]:
# Load in client ID and secret
spotify_id = os.getenv('SpotifyID')
spotify_secret = os.getenv("SpotifySecret")

In [4]:
url = "https://accounts.spotify.com/api/token"
headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}
data = {
    "grant_type": "client_credentials",
    "client_id": spotify_id,
    "client_secret": spotify_secret
}

In [6]:
# Request access token (works for one hour)
r = requests.post(url, headers=headers, data=data)
r

<Response [200]>

In [7]:
access_data = json.loads(r.text)
access_token = access_data['access_token']
token_type = access_data['token_type']

In [8]:
# Load in API keys
genius_id = os.getenv("GeniusID")
genius_secret = os.getenv("GeniusSecret")
genius_token = os.getenv("GeniusToken")
# Get user-agent 
r = requests.get('https://httpbin.org/user-agent')
useragent = json.loads(r.text)['user-agent']
headers = {'User-agent': useragent}

In [9]:
playlist_data_full = pd.read_csv('playlist_data.csv')
playlist_data_full.head()

,Playlist_ID,Playlist_Name,Playlist_Songs,Playlist_Artists,Playlist_Song_IDs
0,6mtYuOxzl58vSGnEDtZ9uB,Pop Hits 2000s – 2025,"['Into You', 'Glad You Came', 'Dark Horse', 'W...","['Ariana Grande', 'The Wanted', 'Katy Perry', ...","['2meEiZKWkiN28gITzFwQo5', '1OXfWI3FQMdsKKC6lk..."
1,34NbomaTu7YuOYnky8nLXL,Pop Hits 2025 (Top 50),"['Die With A Smile', 'APT.', 'Espresso', ""we c...","['Lady Gaga', 'ROSÉ', 'Sabrina Carpenter', 'Ar...","['2plbrEY59IikOBgBGLjaoe', '5vNRhkKd0yEAg8suGB..."
2,4Jb4PDWREzNnbZcOHPcZPy,COUNTRY HITS 2025 🔥 New Country Songs + Top Hits,"['I Had Some Help (Feat. Morgan Wallen)', ""Aus...","['Post Malone', 'Dasha', 'mgk', 'Dylan Marlowe...","['5IZXB5IKAD2qlvTPJYDCFB', '2uqYupMHANxnwgeiXT..."
3,1Cgey68pUlQGsCPI2wJuxr,Best of 2025 🔥 Most Popular Hits 2025 Hits,"['APT.', 'Anxiety', 'Die With A Smile', 'Messy...","['ROSÉ', 'Doechii', 'Lady Gaga', 'Lola Young',...","['5vNRhkKd0yEAg8suGBpjeY', '1musbempyJAw5gfSKZ..."
4,2L2HwKRvUgBv1YetudaRI3,Pop 2000-2010 Bangers,"['Whatcha Say', 'Airplanes (feat. Hayley Willi...","['Jason Derulo', 'B.o.B', 'Bruno Mars', 'Tinch...","['7xkQdy0cy5ymoWT7nedvLz', '1QnvpPFP4Q3FHbDchq..."


In [10]:
import ast
playlist_data_full['Playlist_Songs'] = [ast.literal_eval(x) for x in playlist_data_full['Playlist_Songs']]
playlist_data_full['Playlist_Artists'] = [ast.literal_eval(x) for x in playlist_data_full['Playlist_Artists']]


In [11]:
playlist_data = playlist_data_full.head(10)

In [12]:
genius_url_cache = {}  # {(song_title, artist): genius_url}
lyrics_cache = {}  # {genius_url: lyrics}

In [35]:
import lyricscraper as ls
import importlib

importlib.reload(ls)

playlist_lyrics = []
all_genius_urls = []

for playlist_name, songs, artists in zip(playlist_data['Playlist_Name'], playlist_data['Playlist_Songs'], playlist_data['Playlist_Artists']):
    print(f"Grabbing Genius URLs for playlist: {playlist_name}")
    genius_urls = []
    lyrics_list = []
    # Get Genius URLs for all songs in the playlist
    for song_title, artist in zip(songs, artists):
        if not song_title:
            song_title = ""
        if not artist:
            artist = ""
        song_key = (song_title.lower(), artist.lower())  # Normalize case
        
        if song_key in genius_url_cache:
            url = genius_url_cache[song_key]  # Use cached URL
        else:
            url = ls.get_genius_url(song_title, artist, genius_token)
            genius_url_cache[song_key] = url  # Store in cache
        
        genius_urls.append(url)
        all_genius_urls.append(url)

    # Get lyrics for each song using lyricscraper
    for genius_url in genius_urls:
        if genius_url:
            if genius_url in lyrics_cache:
                lyrics_list.append(lyrics_cache[genius_url])  # Use cached lyrics
            else:
                lyrics = ls.scrape_lyrics(genius_url)
                if not lyrics:
                    lyrics_list.append("Lyrics not found")
                    continue
                lyrics_cache[genius_url] = lyrics  # Store in cache
                lyrics_list.append(lyrics)
        else:
            lyrics_list.append("Lyrics not found")
    
    playlist_lyrics.append(lyrics_list)

# Add collected lyrics to the dataframe
playlist_data = playlist_data.copy()
playlist_data["Playlist_Lyrics"] = playlist_lyrics

Grabbing Genius URLs for playlist: Pop Hits 2000s – 2025
Grabbing Genius URLs for playlist: Pop Hits 2025 (Top 50)
Grabbing Genius URLs for playlist: COUNTRY HITS 2025 🔥 New Country Songs + Top Hits
Grabbing Genius URLs for playlist: Best of 2025 🔥 Most Popular Hits 2025 Hits
Grabbing Genius URLs for playlist: Pop 2000-2010 Bangers
Grabbing Genius URLs for playlist: pop songs everyone knows 🍒
Grabbing Genius URLs for playlist: Top Pop Hits 2015-2025
Grabbing Genius URLs for playlist: Popular Christian Music
Grabbing Genius URLs for playlist: Pop Punk Throwback Bangers - 2000s Emo Punk Rock Hits


In [18]:
playlist_data = playlist_data.copy()
playlist_data["Playlist_Lyrics"] = playlist_lyrics

In [19]:
playlist_data

,Playlist_ID,Playlist_Name,Playlist_Songs,Playlist_Artists,Playlist_Song_IDs,Playlist_Lyrics
0,6mtYuOxzl58vSGnEDtZ9uB,Pop Hits 2000s – 2025,"[Into You, Glad You Came, Dark Horse, Who Knew...","[Ariana Grande, The Wanted, Katy Perry, P!nk, ...","['2meEiZKWkiN28gITzFwQo5', '1OXfWI3FQMdsKKC6lk...","[[Verse 1]\nI'm so into you, I can barely brea..."
1,34NbomaTu7YuOYnky8nLXL,Pop Hits 2025 (Top 50),"[Die With A Smile, APT., Espresso, we can't be...","[Lady Gaga, ROSÉ, Sabrina Carpenter, Ariana Gr...","['2plbrEY59IikOBgBGLjaoe', '5vNRhkKd0yEAg8suGB...","[[Intro: Bruno Mars]\n(Ooh, ooh)\n[Verse 1: Br..."
2,4Jb4PDWREzNnbZcOHPcZPy,COUNTRY HITS 2025 🔥 New Country Songs + Top Hits,"[I Had Some Help (Feat. Morgan Wallen), Austin...","[Post Malone, Dasha, mgk, Dylan Marlowe, Prior...","['5IZXB5IKAD2qlvTPJYDCFB', '2uqYupMHANxnwgeiXT...",[[Verse 1: Post Malone]\nYou got a lotta nerve...
3,1Cgey68pUlQGsCPI2wJuxr,Best of 2025 🔥 Most Popular Hits 2025 Hits,"[APT., Anxiety, Die With A Smile, Messy, Beaut...","[ROSÉ, Doechii, Lady Gaga, Lola Young, Benson ...","['5vNRhkKd0yEAg8suGBpjeY', '1musbempyJAw5gfSKZ...",[[Intro: ROSÉ]\n채영이가 좋아하는 랜덤 게임\n랜덤 게임\nGame s...
4,2L2HwKRvUgBv1YetudaRI3,Pop 2000-2010 Bangers,"[Whatcha Say, Airplanes (feat. Hayley Williams...","[Jason Derulo, B.o.B, Bruno Mars, Tinchy Stryd...","['7xkQdy0cy5ymoWT7nedvLz', '1QnvpPFP4Q3FHbDchq...","[[Intro: Imogen Heap, \nJ.R. Rotem\n]\nWha-wha..."
5,2xutOn4Ea4RyjuaRaD3jl3,pop songs everyone knows 🍒,"[Style (Taylor's Version), traitor, The One Th...","[Taylor Swift, Olivia Rodrigo, Katy Perry, The...","['1hjRhYpWyqDpPahmSlUTlc', '5CZ40GBx1sQ9agT82C...","[[Verse 1]\nMidnight\nYou come and pick me up,..."
6,1WH6WVBwPBz35ZbWsgCpgr,Top Pop Hits 2015-2025,"[Say So, Starboy, Goodbyes (feat. Young Thug),...","[Doja Cat, The Weeknd, Post Malone, DaBaby, Dr...","['3Dv1eDb0MEgF93GpLXlucZ', '7MXVkk9YMctZqd1Srt...","[[Chorus]\nDay to night to morning, keep with ..."
7,327k1ryJ7j1xD7gWKpdc6o,Popular Christian Music,"[Oh My Soul, Stars, Voice of Truth, You Say, I...","[Casting Crowns, Skillet, Casting Crowns, Laur...","['3s0h3pyKFrS3XX6ZLBPx6s', '6VMT3SzIMbNoR5lsUs...","[[Verse 1]\nOh, my soul\nOh, how you worry\nOh..."
8,4ASGb9HSoMEUekd7ClxZxZ,Pop Punk Throwback Bangers - 2000s Emo Punk Ro...,"[Dear Maria, Count Me In, My Friends Over You,...","[All Time Low, New Found Glory, Jimmy Eat Worl...","['0J69x3mqm7U6tBPKsjpsWR', '12mF5rnbbT7jNqiBv8...","[[Intro]\nI got your picture, I'm coming with ..."
9,4muHyvSwG1wP9sI4XihC5w,Instrumental Popular Songs 2025,"[birds of a feather - piano instrumental, gold...","[Chilled Pig, Chilled Pig, Chilled Pig, Edward...","['1bE4hvWlx8gtWIMVUptOXi', '1KYe04kDLflLdoJ9kN...","[Lyrics not found, Lyrics not found, Lyrics no..."


In [21]:
playlist_data.to_csv("Playlist_data_with_lyrics.csv")

In [22]:
playlist_data = playlist_data.head(9)

In [23]:
playlist_data

,Playlist_ID,Playlist_Name,Playlist_Songs,Playlist_Artists,Playlist_Song_IDs,Playlist_Lyrics
0,6mtYuOxzl58vSGnEDtZ9uB,Pop Hits 2000s – 2025,"[Into You, Glad You Came, Dark Horse, Who Knew...","[Ariana Grande, The Wanted, Katy Perry, P!nk, ...","['2meEiZKWkiN28gITzFwQo5', '1OXfWI3FQMdsKKC6lk...","[[Verse 1]\nI'm so into you, I can barely brea..."
1,34NbomaTu7YuOYnky8nLXL,Pop Hits 2025 (Top 50),"[Die With A Smile, APT., Espresso, we can't be...","[Lady Gaga, ROSÉ, Sabrina Carpenter, Ariana Gr...","['2plbrEY59IikOBgBGLjaoe', '5vNRhkKd0yEAg8suGB...","[[Intro: Bruno Mars]\n(Ooh, ooh)\n[Verse 1: Br..."
2,4Jb4PDWREzNnbZcOHPcZPy,COUNTRY HITS 2025 🔥 New Country Songs + Top Hits,"[I Had Some Help (Feat. Morgan Wallen), Austin...","[Post Malone, Dasha, mgk, Dylan Marlowe, Prior...","['5IZXB5IKAD2qlvTPJYDCFB', '2uqYupMHANxnwgeiXT...",[[Verse 1: Post Malone]\nYou got a lotta nerve...
3,1Cgey68pUlQGsCPI2wJuxr,Best of 2025 🔥 Most Popular Hits 2025 Hits,"[APT., Anxiety, Die With A Smile, Messy, Beaut...","[ROSÉ, Doechii, Lady Gaga, Lola Young, Benson ...","['5vNRhkKd0yEAg8suGBpjeY', '1musbempyJAw5gfSKZ...",[[Intro: ROSÉ]\n채영이가 좋아하는 랜덤 게임\n랜덤 게임\nGame s...
4,2L2HwKRvUgBv1YetudaRI3,Pop 2000-2010 Bangers,"[Whatcha Say, Airplanes (feat. Hayley Williams...","[Jason Derulo, B.o.B, Bruno Mars, Tinchy Stryd...","['7xkQdy0cy5ymoWT7nedvLz', '1QnvpPFP4Q3FHbDchq...","[[Intro: Imogen Heap, \nJ.R. Rotem\n]\nWha-wha..."
5,2xutOn4Ea4RyjuaRaD3jl3,pop songs everyone knows 🍒,"[Style (Taylor's Version), traitor, The One Th...","[Taylor Swift, Olivia Rodrigo, Katy Perry, The...","['1hjRhYpWyqDpPahmSlUTlc', '5CZ40GBx1sQ9agT82C...","[[Verse 1]\nMidnight\nYou come and pick me up,..."
6,1WH6WVBwPBz35ZbWsgCpgr,Top Pop Hits 2015-2025,"[Say So, Starboy, Goodbyes (feat. Young Thug),...","[Doja Cat, The Weeknd, Post Malone, DaBaby, Dr...","['3Dv1eDb0MEgF93GpLXlucZ', '7MXVkk9YMctZqd1Srt...","[[Chorus]\nDay to night to morning, keep with ..."
7,327k1ryJ7j1xD7gWKpdc6o,Popular Christian Music,"[Oh My Soul, Stars, Voice of Truth, You Say, I...","[Casting Crowns, Skillet, Casting Crowns, Laur...","['3s0h3pyKFrS3XX6ZLBPx6s', '6VMT3SzIMbNoR5lsUs...","[[Verse 1]\nOh, my soul\nOh, how you worry\nOh..."
8,4ASGb9HSoMEUekd7ClxZxZ,Pop Punk Throwback Bangers - 2000s Emo Punk Ro...,"[Dear Maria, Count Me In, My Friends Over You,...","[All Time Low, New Found Glory, Jimmy Eat Worl...","['0J69x3mqm7U6tBPKsjpsWR', '12mF5rnbbT7jNqiBv8...","[[Intro]\nI got your picture, I'm coming with ..."


In [30]:
lyrics_not_found_count = sum(1 for lyrics in playlist_data["Playlist_Lyrics"][2] if lyrics == ["Lyrics not found"])
lyrics_not_found_count


0

In [33]:
total_songs = sum(len(lyrics_list) for lyrics_list in playlist_data["Playlist_Lyrics"])
total_songs

849

In [34]:
lyrics_not_found_count = sum(lyrics_list.count("Lyrics not found") for lyrics_list in playlist_data["Playlist_Lyrics"])
lyrics_not_found_count

51